In [ ]:
#Upload Compound V2 JSON files
from google.colab import files
import os

uploaded = files.upload()  # Upload all compound*.json files
os.makedirs("compound_v2_raw", exist_ok=True)

# Move uploaded files to a dedicated folder
for name in uploaded.keys():
    os.rename(name, f"compound_v2_raw/{name}")
print("✅ All files moved to /compound_v2_raw")


Saving compoundV2_transactions_ethereum_chunk_2.json to compoundV2_transactions_ethereum_chunk_2.json
Saving compoundV2_transactions_ethereum_chunk_0.json to compoundV2_transactions_ethereum_chunk_0.json
Saving compoundV2_transactions_ethereum_chunk_1.json to compoundV2_transactions_ethereum_chunk_1.json
✅ All files moved to /compound_v2_raw


In [ ]:
#Load and flatten transactions from all JSON files
import json
import pandas as pd
import numpy as np
from glob import glob
from datetime import datetime

json_folder = "compound_v2_raw"
json_files = glob(os.path.join(json_folder, "*.json"))
print(f"🔍 Found {len(json_files)} JSON files.")

if not json_files:
    raise FileNotFoundError("❌ No JSON files found. Please upload them first.")

records = []
for path in json_files:
    with open(path, "r", encoding="utf-8") as f:
        try:
            data = json.load(f)
            for action_type in ["deposits", "borrows", "repays", "withdraws", "liquidations"]:
                for entry in data.get(action_type, []):
                    records.append({
                        "wallet": entry.get("account", {}).get("id", None),
                        "action": action_type[:-1],
                        "amount": float(entry.get("amountUSD", 0)),
                        "timestamp": pd.to_datetime(int(entry.get("timestamp", 0)), unit="s", errors="coerce")
                    })
        except Exception as e:
            print(f"⚠️ Skipped {path}: {e}")

df = pd.DataFrame(records)
df.dropna(subset=["wallet", "timestamp"], inplace=True)
print(f"✅ Loaded {len(df)} normalized transactions.")


🔍 Found 3 JSON files.
✅ Loaded 120000 normalized transactions.


In [ ]:
#Extract wallet-level behavioral features
features = []
wallets = df['wallet'].unique()

for wallet in wallets:
    w_df = df[df['wallet'] == wallet]
    total_txns = len(w_df)
    actions = w_df['action'].value_counts().to_dict()

    deposit_amt = w_df[w_df['action'] == 'deposit']['amount'].sum()
    borrow_amt = w_df[w_df['action'] == 'borrow']['amount'].sum()
    repay_amt = w_df[w_df['action'] == 'repay']['amount'].sum()
    withdraw_amt = w_df[w_df['action'] == 'withdraw']['amount'].sum()
    liquidations = actions.get('liquidation', 0)

    repay_ratio = repay_amt / borrow_amt if borrow_amt > 0 else 0
    deposit_borrow_ratio = deposit_amt / borrow_amt if borrow_amt > 0 else 0
    net_change = deposit_amt - withdraw_amt
    days_active = (w_df['timestamp'].max() - w_df['timestamp'].min()).days + 1
    txn_frequency = total_txns / days_active if days_active > 0 else 0

    features.append({
        'wallet': wallet,
        'total_txns': total_txns,
        'deposit_amt': deposit_amt,
        'borrow_amt': borrow_amt,
        'repay_amt': repay_amt,
        'withdraw_amt': withdraw_amt,
        'repay_ratio': repay_ratio,
        'deposit_borrow_ratio': deposit_borrow_ratio,
        'net_change': net_change,
        'liquidations': liquidations,
        'txn_frequency': txn_frequency
    })

wallet_df = pd.DataFrame(features)


In [ ]:
#Normalize features and compute credit score
def normalize(series):
    return (series - series.min()) / (series.max() - series.min() + 1e-9)

wallet_df['score'] = (
    normalize(wallet_df['repay_ratio']) * 0.4 +
    normalize(wallet_df['deposit_borrow_ratio']) * 0.2 +
    normalize(wallet_df['txn_frequency']) * 0.15 +
    normalize(wallet_df['net_change']) * 0.15 +
    (1 - normalize(wallet_df['liquidations'])) * 0.1
) * 100

wallet_df['score'] = wallet_df['score'].round(2)
wallet_df = wallet_df.sort_values(by='score', ascending=False)


In [ ]:
#Save top 1000 scores to CSV
top_1000 = wallet_df[['wallet', 'score']].head(1000)
top_1000.to_csv("top_1000_wallet_scores.csv", index=False)
print("📁 Saved: top_1000_wallet_scores.csv")

# 🔽 Step 6: Download CSV to your local machine
from google.colab import files
files.download("top_1000_wallet_scores.csv")


📁 Saved: top_1000_wallet_scores.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Display wallet summaries for 1-pager
print("\n🟢 Top 5 High-Scoring Wallets:")
print(wallet_df[['wallet', 'score', 'repay_ratio', 'liquidations']].head(5))

print("\n🔴 Bottom 5 Low-Scoring Wallets:")
print(wallet_df[['wallet', 'score', 'repay_ratio', 'liquidations']].tail(5))



🟢 Top 5 High-Scoring Wallets:
                                           wallet  score  repay_ratio  \
11267  0xc57f71d926d86a2d8e36829126e2aa3db6784212  55.94   218.082163   
1579   0x920f2f74cf3f3c04e9bf79fbe699889a3d153295  35.25     0.000000   
4161   0xe705751e4263848613ecb13b04e398cee7d8eca5  29.17     0.538739   
3877   0x136d9aade0fabb66dfaf7fba1d450c447d8d9d3d  29.15     0.442166   
554    0x1e105686de9d256eb107e1c25114088076199bde  28.59     0.559520   

       liquidations  
11267             0  
1579              0  
4161              0  
3877              0  
554               0  

🔴 Bottom 5 Low-Scoring Wallets:
                                           wallet  score  repay_ratio  \
2820   0x1cf1b39d9db93bd13b41f0d5be32b240dcd002ad  12.64     0.000000   
2594   0x794dec90a3b9e399aad7839fa39c17a521977cdf  12.33     0.940531   
10305  0xc9c5a0228fb686e9720ca24691e59e9410c52688  12.20     0.000000   
11843  0xea5ee32f3a63c3fabb311c6e8c985d308a53dcc1  11.39     0.000000   
